# Code Examples

#### Examples of using functions from `key_functions.py` 

In [ ]:
import key_functions as kf

import math
import pickle
import pandas as pd

from sklearn.linear_model import LogisticRegression

###### Classifier Training

In [ ]:
# Training for damages
dmg_model, dmg_vectorizer = kf.train_classifier('../data/annotations/all_annotations.txt', clf = LogisticRegression(solver = 'newton-cg', C = 1, penalty = 'l2'))

# Training for contributory negligence
cn_model, cn_vectorizer = kf.train_CN_classifier('../data/annotations/all_annotations.txt', clf = LogisticRegression(solver = 'newton-cg', C = 1, penalty = 'l2'))


###### Saving & Loading Claasifier

Loading classifier and vectorizer from `.pkl` format 

In [ ]:
# Save Code
# WARNING: This will overwrite the currently saved model. Uncomment lines as needed.

# with open('./best_model.pkl', 'wb') as file:
#     pickle.dump(dmg_model, file)
    
# with open('./best_vectorizer.pkl', 'wb') as file:
#     pickle.dump(dmg_vectorizer, file)
    
# with open('./best_CN_model.pkl', 'wb') as file:
#     pickle.dump(cn_model, file)
    
# with open('./best_CN_vectorizer.pkl', 'wb') as file:
#     pickle.dump(cn_vectorizer, file)

In [ ]:
# Load Code
with open('./best_model.pkl', 'rb') as file:
    dmg_model = pickle.load(file)
    
with open('./best_vectorizer.pkl', 'rb') as file:
    dmg_vectorizer = pickle.load(file)
    
with open('./best_CN_model.pkl', 'rb') as file:
    cn_model = pickle.load(file)
    
with open('./best_CN_vectorizer.pkl', 'rb') as file:
    cn_vectorizer = pickle.load(file)

###### Using the classifier

In [ ]:
path_to_data = '../data/Lexis Cases txt/'
file_prefix = 'P'
file_suffix = '.txt'
file_identifiers = range(1, 86) # Range from 1 to 85

clf_results = []
for file_number in file_identifiers:
    print('## Processing ' + path_to_data + file_prefix + str(file_number) + file_suffix + ' ##', end='\r')
    
    # Pass the model to the rule_based_parse_BCJ function
    # This will use rule based for columns such as judge name, etc. but classifier for damages/CN
    
    # To use rule based for damage/CN do not pass the model/vectorizer.
    clf_results.extend(kf.rule_based_parse_BCJ(path_to_data + file_prefix + str(file_number) + file_suffix, dmg_model, dmg_vectorizer))
    

###### Evaluating the model

In [ ]:
gold_df = pd.read_csv('../data/gold_annotations.csv')#, #skiprows = lambda x: x in range(1, 100))
gold_df.dropna(how = 'all', inplace=True) 

dev_df = kf.rule_based_convert_cases_to_DF(clf_results)
kf.evaluate(dev_df, gold_df)